In [81]:
from sklearn.datasets import make_blobs
from pathlib import Path
from sqlalchemy import create_engine
from pathlib import Path
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
from pprint import pprint
import json
from citipy import citipy
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from datetime import datetime
warnings.filterwarnings("ignore")

In [92]:
# load csv file
file_path = pd.read_csv('/Users/pedrosiqueira/Desktop/Project4/Update_Listings_cleaned_v1.csv')
# create a dataframe from the loaded csv file
cleaned_data_df = pd.DataFrame(file_path)
# print out the first 5 rows of the listing_df DataFrame
cleaned_data_df.head()

,unique_identifier,listing_amount,amount_funded,lender_yield,borrower_apr,listing_term,listing_monthly_payment,income_range,income_range_description,stated_monthly_income,...,lender_indicator,group_indicator,Fico_rating,TUFicoRange,dti_wprosper_loan,CoBorrowerApplication,CombinedDtiwProsperLoan,CombinedStatedMonthlyIncome,rating_num,prosper_rating
0,0,17300,17300,0.1349,0.1683,60,406.95,6,"$100,000+",11250.0000,...,0,False,6,760-779,0.2377,True,0.2377,16250.0000,1,B
1,1,25000,25000,0.0999,0.1325,60,543.44,6,"$100,000+",15416.6667,...,0,False,6,760-779,0.2900,True,0.2900,22333.3333,0,A
2,2,3500,3500,0.1179,0.1641,36,117.58,4,"$50,000-74,999",4166.6667,...,0,False,3,700-719,0.0309,True,0.0309,7500.0000,1,B
3,3,5000,5000,0.0964,0.1422,36,162.84,4,"$50,000-74,999",5000.0000,...,0,False,1,660-679,0.1818,True,0.1818,8333.3333,0,A
4,4,21795,21795,0.0530,0.0795,36,666.01,6,"$100,000+",20000.0000,...,0,False,9,820-850,0.1816,True,0.1816,23833.3333,0,AA


In [93]:
integer_df = cleaned_data_df.astype(int,errors='ignore')
integer_df

,unique_identifier,listing_amount,amount_funded,lender_yield,borrower_apr,listing_term,listing_monthly_payment,income_range,income_range_description,stated_monthly_income,...,lender_indicator,group_indicator,Fico_rating,TUFicoRange,dti_wprosper_loan,CoBorrowerApplication,CombinedDtiwProsperLoan,CombinedStatedMonthlyIncome,rating_num,prosper_rating
0,0,17300,17300,0,0,60,406,6,"$100,000+",11250,...,0,0,6,760-779,0,1,0,16250,1,B
1,1,25000,25000,0,0,60,543,6,"$100,000+",15416,...,0,0,6,760-779,0,1,0,22333,0,A
2,2,3500,3500,0,0,36,117,4,"$50,000-74,999",4166,...,0,0,3,700-719,0,1,0,7500,1,B
3,3,5000,5000,0,0,36,162,4,"$50,000-74,999",5000,...,0,0,1,660-679,0,1,0,8333,0,A
4,4,21795,21795,0,0,36,666,6,"$100,000+",20000,...,0,0,9,820-850,0,1,0,23833,0,AA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,656,7500,7500,0,0,36,248,6,"$100,000+",10000,...,0,0,3,700-719,0,1,0,14333,0,A
657,657,35000,35000,0,0,60,792,4,"$50,000-74,999",4833,...,0,0,6,760-779,0,1,0,10750,1,B
658,658,15000,15000,0,0,60,334,6,"$100,000+",10825,...,0,0,2,680-699,0,1,0,14705,0,A
659,659,15000,15000,0,0,60,339,6,"$100,000+",26325,...,0,0,3,700-719,0,1,0,27575,1,B


In [84]:
# Split the data into X (features) and y (target)
# The y variable should focus on the target column
y = integer_df['rating_num']

# The X variable should include all features except the target
X = integer_df.drop([
    "listing_amount", "unique_identifier","lender_yield", "borrower_apr", "listing_monthly_payment",
    "income_range_description","employment_status_description", "occupation", 
    "borrower_state_num", "borrower_state", "borrower_city", "group_indicator", 
    "TUFicoRange", "CoBorrowerApplication", "rating_num", "prosper_rating"    
], axis=1)



In [85]:
X.head()

,amount_funded,listing_term,income_range,stated_monthly_income,employment_status,months_employed,prior_prosper_loans,prior_prosper_loans_principal_borrowed,prior_prosper_loans_principal_outstanding,prior_prosper_loans_balance_outstanding,prior_prosper_loans_cycles_billed,prior_prosper_loans_ontime_payments,prior_prosper_loans_late_cycles,prior_prosper_loan_earliest_pay_off,lender_indicator,Fico_rating,dti_wprosper_loan,CombinedDtiwProsperLoan,CombinedStatedMonthlyIncome
0,17300,60,6,11250,0,300,2,25000,8809,0,1,1,0,7,0,6,0,0,16250
1,25000,60,6,15416,0,32,2,17000,0,0,0,0,0,1,0,6,0,0,22333
2,3500,36,4,4166,0,11,3,15000,0,0,15,15,0,0,0,3,0,0,7500
3,5000,36,4,5000,0,6,1,3000,0,0,0,0,0,24,0,1,0,0,8333
4,21795,36,6,20000,0,57,1,19000,0,0,0,0,0,30,0,9,0,0,23833


In [86]:
y.head()

0    1
1    0
2    1
3    0
4    0
Name: rating_num, dtype: int64

In [87]:
# Split into testing and training sets using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

In [88]:

# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [89]:
# make prediction on the test set
y_prediction = model.predict(X_test)
print(y_prediction)


[ 0.69164067  0.79183722  0.47701043  0.32408298  0.40408642  1.18228572
  0.14383053  0.16428735  0.80541978  0.45350199  0.26725175  0.05988237
  0.11942281  0.37230155  0.93761305 -0.02393366  0.49284148  0.43175136
  0.30571691  0.74919861  0.86008098  0.23787295  0.27049147  0.67632339
  0.81888388  0.81222366  0.09167624 -0.25857589  1.20259798  0.05321004
  0.04108862  0.68465172  0.65487033  0.34297702  0.59589803  0.52423037
  0.42473471  0.42034295  0.06661296  0.46849796  0.4560152  -0.05141466
  0.15164678  0.7234913   0.71544986 -0.05246179  0.86783097  0.10742329
  0.22254347  1.15653188  0.72850248  0.30301494  0.52967047  0.71494508
  0.37249345 -0.02532792  0.28590104  0.76781836  0.4493713   0.68006024
 -0.1221626   0.75095068 -0.11214261  0.6685905   0.96013108  0.53769921
  0.65867714  0.53618463  0.05580938  0.2684942   0.74344546  0.14616846
  0.90498474  0.64420299  0.50378845  0.67950473  0.43049808  1.23940402
  0.2807135   0.27729874 -0.18753443  0.18215057 -0

In [90]:
# logistical regression

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions using the model
y_prediction = model.predict(X_test)

# Compute metrics
accuracy = accuracy_score(y_test, y_prediction)
confusion_matrix = confusion_matrix(y_test, y_prediction)
classification_report = classification_report(y_test, y_prediction)

# Print relevant metrics
print(f"The accuracy is {accuracy}.")
print(f"The confusion matrix is {confusion_matrix}.")
print(f"The classification report is {classification_report}.")

The accuracy is 0.631578947368421.
The confusion matrix is [[73 10  0]
 [35 11  0]
 [ 4  0  0]].
The classification report is               precision    recall  f1-score   support

           0       0.65      0.88      0.75        83
           1       0.52      0.24      0.33        46
           2       0.00      0.00      0.00         4

    accuracy                           0.63       133
   macro avg       0.39      0.37      0.36       133
weighted avg       0.59      0.63      0.58       133
.


In [91]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions using the model
y_prediction = model.predict(X)

# Compute metrics
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, y_prediction)
mse = mean_squared_error(y, y_prediction)
rmse = np.sqrt(mse)
std = np.std(y_prediction)

# Print relevant metrics
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.6520423600605144.
The r2 is -0.30174673639561056.
The mean squared error is 0.4523449319213313.
The root mean squared error is 0.6725659312820798.
The standard deviation is 0.36683640704747333.
